In [1]:
from llm import OpenAICompatibleChatCompletion, LLModel
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from ecommerce import questions, ecommerce_metrics, sales_channels, brands
import numpy as np
import json

In [2]:
llm = OpenAICompatibleChatCompletion()
llm.call(messages=[{"role": "system", "content": "Talk like a pirate and return JSON"}, {"role": "user", "content": "hey"}], **LLModel.GPT3_0125)

{'message': {'content': "Ahoy matey! What be ye needin' help with, me heartie?",
  'role': 'assistant'},
 'model': 'gpt-3.5-turbo-0125'}

In [ ]:
def prepare_embedding_input(rec):
    return f'{rec["name"]}\n{rec["enum"]}'


ecommerce_metrics_embeddings = llm.get_embeddings([prepare_embedding_input(rec) for rec in ecommerce_metrics])
brands_embeddings = llm.get_embeddings([prepare_embedding_input(rec) for rec in brands])

In [ ]:
get_backend_metric_tool = {
    "type": "function",
    "function": {
        "name": "get_backend_metric",
        "description": """Takes in the user requested metric and 
        uses ML/AI to return the k nearest neighbors for the most likely related backend ENUM metrics.""",
        "parameters": {
            "type": "object",
            "properties": {
                "user_requested_metric": {
                    "type": "string",
                    "description": "The metric requested by the user.",
                },
            },
            "required": ["user_requested_metric"],
        },
    },
}
get_backend_brands_tool = {
    "type": "function",
    "function": {
        "name": "get_backend_brands",
        "description": """Takes in the user requested brand(s) and 
        uses ML/AI to return the k nearest neighbors for the most likely related backend ENUM brands per requested brands.""",
        "parameters": {
            "type": "object",
            "properties": {
                "user_requested_brands": {
                    "type": "array",
                    "items": {
                        "type": "string",
                    },
                    "default": [],
                    "description": "The list of brand(s) requested by the user.",
                },
            },
            "required": ["user_requested_brands"],
        },
    },
}
get_sales_data_tool = {
    "type": "function",
    "function": {
        "name": "get_sales_data",
        "description": """Get the sales data from the backend system.""",
        "parameters": {
            "type": "object",
            "properties": {
                "backend_metric": {
                    "type": "string",
                    "description": "This is the backend metric ENUM.",
                },
                "backend_brands": {
                    "type": "array",
                    "items": {
                        "type": "string",
                    },
                    "default": [],
                    "description": "The list of backend ENUM brands.",
                },
                "sales_channels": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "enum": [x["enum"] for x in sales_channels],
                    },
                    "default": [],
                    "description": "The list of sales channels.",
                },
                "current_period_start_date": {
                    "type": "string",
                    "description": "The start of the current reporting period.",
                },
                "current_period_end_date": {
                    "type": "string",
                    "description": "The end of the current reporting period.",
                },
            },
            "required": [
                "backend_metric",
                "backend_brands",
                "current_period_start_date",
                "current_period_end_date",
            ],
        },
    },
}
tools = [get_backend_metric_tool, get_backend_brands_tool, get_sales_data_tool]

In [ ]:
def find_k_nearest_neighbors(embeddings, input_embedding, k):
    # Calculate distances
    distances = np.linalg.norm(embeddings - input_embedding, axis=1)

    # Get indices of k smallest distances
    nearest_indices = np.argpartition(distances, k)[:k]

    # Sort the k nearest indices by distance
    nearest_indices = nearest_indices[np.argsort(distances[nearest_indices])]

    return nearest_indices

In [ ]:
def get_backend_metric(user_requested_metric: str):
    return {
        "data": [ecommerce_metrics[i] for i in find_k_nearest_neighbors(llm.get_embeddings([user_requested_metric]), ecommerce_metrics_embeddings, 3)]
    }


def get_backend_brands(user_requested_brands: list[str]):
    data = dict()
    for brand in user_requested_brands:
        data[brand] = [brands[i] for i in find_k_nearest_neighbors(llm.get_embeddings([brand]), brands_embeddings, 3)]
    return {"data": data}


def get_sales_data(*args, **kwargs):
    return {"data": 10}


functions_look_up = {"get_backend_metric": get_backend_metric, "get_backend_brands": get_backend_brands, "get_sales_data": get_sales_data}

In [ ]:
get_backend_brands(["shopify", "nike"])

In [ ]:
system_prompt = """
You will be asked a question by the user about retrieving sales data.
Use the available tools but only call the tools when needed.
If you need further clarification then ask. 

There are hundreds of metrics and hundreds of brands in the backend system.
The user will not know all these metrics and brands or how to refer to them exactly.
You do not know them all of them either, so I have provided some helper tools for you.

In general you will follow the typical flow when answering questions:
1. Extract the user requested metric and the user requested brand(s).
2. 
    
    a) Pass the user requested metric to the the tool get_backend_metric to
    get the list of most likely corresponding backend metric ENUMs. 
    Then choose the most appropriate from this list. 
    
    b) Pass the user requested brand(s) to the the tool get_backend_brands to
    get the list of most likely corresponding backend brand ENUMs. 
    Then choose the most appropriate from this list. 
    
3. Extract any sales channels if mentioned.
4. Pass the relevant arguments into the get_sales_data tool.


Today's date is Monday, June 10, 2024
"""

In [ ]:
def eval_llm_resp(question: dict, llm_resp: dict):
    if not llm_resp.get("tool_calls_details"):
        args_predicted = dict()
    else:
        args_predicted = [
            json.loads(x["function"]["arguments"]) for x in llm_resp["tool_calls_details"].values() if x["function"]["name"] == "get_sales_data"
        ][0]
    return {
        "question": question["question"],
        "expected_metric": question["expected_metric"],
        "predicted_metric": args_predicted.get("backend_metric", ""),
        "metric_correct": question["expected_metric"] == args_predicted.get("backend_metric", ""),
        "expected_brands": sorted(question["expected_brands"]),
        "predicted_brands": sorted(args_predicted.get("backend_brands", [])),
        "brands_correct": sorted(question["expected_brands"]) == sorted(args_predicted.get("backend_brands", [])),
        "expected_sales_channels": sorted(question["expected_sales_channels"]),
        "predicted_sales_channels": sorted(args_predicted.get("sales_channels", [])),
        "sales_channels_correct": sorted(question["expected_sales_channels"]) == sorted(args_predicted.get("sales_channels", [])),
        "expected_current_period_start_date": question["current_period_start_date"],
        "predicted_current_period_start_date": args_predicted.get("current_period_start_date", ""),
        "current_period_start_date_correct": question["current_period_start_date"] == args_predicted.get("current_period_start_date", ""),
        "expected_current_period_end_date": question["current_period_end_date"],
        "predicted_current_period_end_date": args_predicted.get("current_period_end_date", ""),
        "current_period_end_date_correct": question["current_period_end_date"] == args_predicted.get("current_period_end_date", ""),
    }


def eval_questions(questions: list[dict]):
    def task(question: dict):
        llm_resp = OpenAICompatibleChatCompletion.generate_with_function_calling(
            messages=[
                {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": question["question"],
                },
            ],
            tools=tools,
            functions_look_up=functions_look_up,
            **LLModel.GPT3_0125,
        )
        llm_resp.update(eval_llm_resp(question, llm_resp))
        return llm_resp

    eval_res = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(task, question) for question in questions]

        for future in tqdm(as_completed(futures), total=len(questions), desc="Evaluating questions"):
            eval_res.append(future.result())

    return eval_res

In [ ]:
import pandas as pd

df = pd.DataFrame(eval_questions(questions[:40]))

In [ ]:
df

In [ ]:
def calculate_accuracies(df):
    accuracies = {}
    for col in df.columns:
        if col.endswith("_correct"):
            accuracy = df[col].sum() / df.shape[0]
            accuracies[col.replace("correct", "accuracy")] = f"{accuracy:.2%}"
    return accuracies


accuracies = calculate_accuracies(df)
print(accuracies)

In [ ]:
df[~df["sales_channels_correct"]]

In [ ]:
import anthropic

client = anthropic.Anthropic()
model = "claude-3-5-sonnet-20240620"
message = client.messages.create(
    model=model,
    max_tokens=1000,
    temperature=0,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[{"role": "user", "content": "Why is the ocean salty?"}, {"role": "assistant", "content": "No, as an AI"}],
)
print(message)

m = {"role": "user", "content": "I'm Jeremy"}
r = client.messages.create(messages=[m], model=model, max_tokens=100)
r